# 📊 EDA (Exploratory Data Analysis)

## 目的
- データの基本構造を理解
- 特徴量の分布・相関を分析
- 欠損値・外れ値の特定
- モデリング方針の決定

In [ ]:
# 基本ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 設定
plt.style.use('default')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# データ読み込み
# TODO: ファイルパスを適切に設定
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')

print(f"Train shape: {df_train.shape}")
print(f"Test shape: {df_test.shape}")

## 1. データ概要

In [ ]:
# 基本情報
display(df_train.head())
display(df_train.info())
display(df_train.describe())

## 2. 欠損値分析

In [ ]:
# 欠損値の可視化
def plot_missing_values(df, title):
    missing = df.isnull().sum()
    missing_pct = missing / len(df) * 100
    missing_df = pd.DataFrame({
        'column': missing.index,
        'missing_count': missing.values,
        'missing_percentage': missing_pct.values
    })
    missing_df = missing_df[missing_df['missing_count'] > 0].sort_values('missing_count', ascending=False)
    
    if len(missing_df) > 0:
        fig = px.bar(missing_df, x='column', y='missing_percentage', 
                     title=f'{title} - Missing Values (%)')
        fig.show()
    else:
        print(f"{title}: 欠損値なし")
    
    return missing_df

missing_train = plot_missing_values(df_train, 'Train')
missing_test = plot_missing_values(df_test, 'Test')

## 3. ターゲット変数分析

In [ ]:
# TODO: ターゲット変数名を適切に設定
target_col = 'target'  # 実際のターゲット変数名に変更

if target_col in df_train.columns:
    # ターゲット変数の分布
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # ヒストグラム
    df_train[target_col].hist(bins=50, ax=axes[0])
    axes[0].set_title('Target Distribution')
    axes[0].set_xlabel(target_col)
    
    # 統計情報
    axes[1].text(0.1, 0.8, f"Mean: {df_train[target_col].mean():.4f}")
    axes[1].text(0.1, 0.7, f"Std: {df_train[target_col].std():.4f}")
    axes[1].text(0.1, 0.6, f"Min: {df_train[target_col].min():.4f}")
    axes[1].text(0.1, 0.5, f"Max: {df_train[target_col].max():.4f}")
    axes[1].text(0.1, 0.4, f"Skew: {df_train[target_col].skew():.4f}")
    axes[1].text(0.1, 0.3, f"Kurtosis: {df_train[target_col].kurtosis():.4f}")
    axes[1].set_xlim(0, 1)
    axes[1].set_ylim(0, 1)
    axes[1].set_title('Target Statistics')
    
    plt.tight_layout()
    plt.show()
else:
    print("ターゲット変数名を確認してください")

## 4. 特徴量分析

In [ ]:
# 数値特徴量とカテゴリ特徴量の分離
numeric_features = df_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = df_train.select_dtypes(include=['object']).columns.tolist()

# ターゲット変数を除外
if target_col in numeric_features:
    numeric_features.remove(target_col)

print(f"数値特徴量: {len(numeric_features)}個")
print(f"カテゴリ特徴量: {len(categorical_features)}個")

In [ ]:
# 数値特徴量の分布
if len(numeric_features) > 0:
    n_cols = 4
    n_rows = (len(numeric_features) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5 * n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 else axes
    
    for i, feature in enumerate(numeric_features[:len(axes)]):
        df_train[feature].hist(bins=50, ax=axes[i])
        axes[i].set_title(f'{feature}')
        axes[i].set_xlabel(feature)
    
    # 余ったサブプロットを非表示
    for i in range(len(numeric_features), len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()

## 5. 相関分析

In [ ]:
# 数値特徴量の相関マトリックス
if len(numeric_features) > 1:
    corr_features = numeric_features + ([target_col] if target_col in df_train.columns else [])
    correlation_matrix = df_train[corr_features].corr()
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
                square=True, fmt='.2f')
    plt.title('Feature Correlation Matrix')
    plt.tight_layout()
    plt.show()
    
    # ターゲットとの相関（高い順）
    if target_col in df_train.columns:
        target_corr = correlation_matrix[target_col].abs().sort_values(ascending=False)
        print("\nターゲットとの相関（絶対値、高い順）:")
        print(target_corr.head(10))

## 6. カテゴリ特徴量分析

In [ ]:
# カテゴリ特徴量の一意値数
if len(categorical_features) > 0:
    cat_info = pd.DataFrame({
        'feature': categorical_features,
        'unique_count': [df_train[col].nunique() for col in categorical_features],
        'missing_count': [df_train[col].isnull().sum() for col in categorical_features]
    })
    cat_info = cat_info.sort_values('unique_count', ascending=False)
    display(cat_info)
    
    # 高カーディナリティ特徴量の確認
    high_cardinality = cat_info[cat_info['unique_count'] > 50]['feature'].tolist()
    if high_cardinality:
        print(f"\n高カーディナリティ特徴量 (>50): {high_cardinality}")

## 7. 外れ値検出

In [ ]:
# 数値特徴量の外れ値検出（IQR法）
if len(numeric_features) > 0:
    outlier_info = []
    
    for feature in numeric_features:
        Q1 = df_train[feature].quantile(0.25)
        Q3 = df_train[feature].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = df_train[(df_train[feature] < lower_bound) | (df_train[feature] > upper_bound)]
        outlier_count = len(outliers)
        outlier_percentage = outlier_count / len(df_train) * 100
        
        outlier_info.append({
            'feature': feature,
            'outlier_count': outlier_count,
            'outlier_percentage': outlier_percentage,
            'lower_bound': lower_bound,
            'upper_bound': upper_bound
        })
    
    outlier_df = pd.DataFrame(outlier_info)
    outlier_df = outlier_df.sort_values('outlier_percentage', ascending=False)
    display(outlier_df.head(10))

## 8. 次のステップ

### 発見した特徴
- TODO: EDA結果をまとめる

### 前処理が必要な項目
- TODO: 欠損値処理
- TODO: 外れ値処理
- TODO: 特徴量エンジニアリング

### モデリング方針
- TODO: 適用予定のアルゴリズム
- TODO: 評価指標の確認
- TODO: 交差検証戦略